# Classification
Implement k-nearest neighbours using any programming language of your choice. Do
not use any library such as scikit-learn that already has k-nearest neighbour or cross validation implemented.
Implementing k-nearest neighbour and cross validation from scratch will be a good exercise to make sure that
you fully understand those algorithms. Feel free to use general libraries for array and matrix operations such as
numpy. Feel free to verify the correctness of your implementation with existing libraries such as scikit-learn.

Download the dataset posted on the course web page. Classify each input x according to the most frequent
class amongst its k nearest neighbours as measured by the Euclidean distance (L2-norm). Break ties at random.
Determine the best number of neighbours k by 10-fold cross validation.

- Dataset for K-nearest neighbour: knn-dataset.zip

    Problem: this data is a modified version of the Optical Recognition of Handwritten Digits Dataset from the UCI repository. It contains pre-processed black and white images of the digits 5 and 6. Each attribute indicates how many pixels are black in a patch of 4 x 4 pixels.
- Format: there is one row per image and one column per attribute. The class labels are 5 and 6.
- The training set is already divided into 10 subsets for 10-fold cross validation.

# Ideas for optimization
- kMeans

    [A fast exact k-nearest neighbors algorithm for high dimensional search using k-means clustering and triangle inequality](https://ieeexplore.ieee.org/abstract/document/6033373)
- Ball-Trees

    [Efficient Exact k-NN and Nonparametric
Classification in High Dimensions](http://www.cs.cmu.edu/afs/cs/Web/People/tingliu/my_papers/nips03.pdf)
- Kd-Trees
    
    [kd-trees](https://en.wikipedia.org/wiki/K-d_tree)

# TensorFlow Numpy incompatibilities / missing features

1. np.linalg does not exist
```python
tf.norm()
# Instead of
np.linalg.norm()
```
2. ndarray.sum does not exist, use np.sum
3. module `tensorflow.experimental.numpy.random` has no attribute `choice()`, use: 
```python
tnp.random.randint
```
4. 'ndarray' object has no attribute 'nbytes'
```python
x.data.numpy().nbytes
```
5. 'ndarray' object has no attribute 'argpartition'
- Numpy
```python
nearest_neighbors_index = distances.T.argpartition(kth=k)[:, :k]
```
- TensorFlow
```python
nearest_neighbors_index = tf.math.top_k(-distances.T, k=k).indices
```
6. TensorFlow has no errstate and issues no warning on inf or nan (np.errstate is not needed)
7. TF "in" like operator:
```
alpha in [22, 28]
# In TF
tf.reduce_any(alpha == tf.constant([22, 28]))
``` 


In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import sklearn as skl
from sklearn import preprocessing

import tensorflow as tf
import tensorflow.experimental.numpy as tnp
from  tensorflow.python.ops.numpy_ops import np_config 
import tensorboard

import plotly.graph_objects as go

In [2]:
# tensorflow.experimental.numpy.experimental_enable_numpy_behavior only exists in tf-nightly.
np_config.enable_numpy_behavior()

In [3]:
%load_ext tensorboard

# Util

In [4]:
def read_csv(file_name_prefix, partition=''):
    # np.genfromtxt('trainData1.csv', dtype=int, delimiter=',')
    return pd.read_csv(f'{file_name_prefix}{partition}.csv', header=None, dtype=np.float64).values

def read_train_data(num_partitions):
    train_data = [read_csv('trainData', i + 1) for i in range(num_partitions)]
    train_labels = [read_csv('trainLabels', i + 1)[:, 0] for i in range(num_partitions)]
    return train_data, train_labels

def read_test_data():
    test_data = read_csv('testData')
    test_labels = read_csv('testLabels')[:, 0]
    return test_data, test_labels

def norm(v):
    norm_v = np.linalg.norm(v, axis=1)[:, np.newaxis]
    norm_v[norm_v == 0] = 1
    return v / norm_v

@tf.function(
    input_signature=[
                    tf.TensorSpec(shape=None, dtype=tf.float64), 
                    tf.TensorSpec(shape=None, dtype=tf.float64), 
                    tf.TensorSpec(shape=(), dtype=tf.int32),
                    tf.TensorSpec(shape=(), dtype=tf.int32),
                    ]
            )
def split_train_validation_tf(x, y, fold, folds):
    """
    Splits input data into train and validation sets.
    Used in k-fold cross validation.
    """
    print("tracing split_train_validation_tf.")
    n = tf.shape(x)[0]
    fold_size = n // folds
    fold_start = fold * fold_size
    fold_end = (fold + 1) * fold_size
    train_x = tf.concat([x[:fold_start], x[fold_end:]], axis=0)
    train_y = tf.concat([y[:fold_start], y[fold_end:]], axis=0)
    validation_x, validation_y = x[fold_start:fold_end], y[fold_start:fold_end]
    return train_x, train_y, validation_x, validation_y

In [5]:
def plot_alpha_scores(alphas, scores, title='Alpha Scores'):
    max_score = scores.argmax()
    alpha_scores_fig = go.Figure()
    alpha_scores_fig.add_trace(go.Scatter(x=alphas, y=scores, mode='lines', name='Alpha Scores'))
    alpha_scores_fig.add_trace(go.Scatter(x=[alphas[max_score]], y=[scores[max_score]], mode='markers', name='Best Alpha'))
    alpha_scores_fig.update_layout(title=title, autosize=True, width=500, height=500,)
    alpha_scores_fig.update_xaxes(title_text='Alpha')
    alpha_scores_fig.update_yaxes(title_text='Score')
    alpha_scores_fig.show()


# Reading training / test data

In [29]:
xtrp, ytrp = read_train_data(num_partitions=10)

In [30]:
xtrp = [norm(x) for x in xtrp]

In [31]:
xtr, ytr = np.concatenate(xtrp), np.concatenate(ytrp)

In [32]:
xte, yte = read_test_data()

In [33]:
xte = norm(xte)

In [34]:
xtr.shape, ytr.shape, xte.shape, yte.shape, type(xtr[0][0]), type(xte[0][0])

((1000, 64), (1000,), (110, 64), (110,), numpy.float64, numpy.float64)

In [74]:
class KNearestNeighborsClassifierTF:
    """
    Brute force implementation of K nearest neighbors.
    
    Introduces preditiction of multiple points using matrix operations (no iteration)
    Runs into OOM if too many data points are used. (need to implement batching in "predict")
    """
    def __init__(self, k=None, xtr=None, ytr=None):
        self.k = k
        self.xtr = xtr
        self.ytr = ytr
        self.k_scores = None
    
    def fit(self, X, y, alphas, folds=10):
        k, k_scores = self.fit_tf(X, y, alphas, folds)
        self.xtr, self.ytr = X, y
        self.k, self.k_scores = k.numpy(), k_scores.numpy()

    @tf.function(
        input_signature=[
                         tf.TensorSpec(shape=None, dtype=tf.float64), 
                         tf.TensorSpec(shape=None, dtype=tf.float64), 
                         tf.TensorSpec(shape=None, dtype=tf.int32),
                         tf.TensorSpec(shape=(), dtype=tf.int32),
                         ]
                 )
    def fit_tf(self, X, y, alphas, folds):
        """
        Trains model using cross validation.
        """
        alphas_size = tf.size(alphas)
        scores = tf.TensorArray(tf.float64, size=alphas_size)
        scores = scores.unstack(tf.zeros_like(alphas, dtype=tf.float64))

        for fold in tf.range(folds, dtype=tf.int32):
            # splits train and validation sets
            xtr, ytr, xvl, yvl = split_train_validation_tf(X, y, fold, folds)
            for i in tf.range(alphas_size):
                # lazy learner...
                alpha = alphas[i]
                # calculates score using validation set
                score = self.score_tf(xvl, yvl, xtr, ytr, alpha)
                # accumulates score of each alpha over all folds
                scores = scores.write(i, scores.read(i) + score)
        
        alpha_scores = scores.stack() / folds
        # best alpha has the highest score sum over all folds
        alpha = alphas[tf.argmax(alpha_scores)][0]
        return alpha, alpha_scores         
    
    def predict(self, X):
        assert self.xtr is not None and self.ytr is not None and self.k, "k, X and y must be provided at creation or you must use the 'fit' method."
        return self.predict_tf(X, self.xtr, self.ytr, self.k)

    @tf.function(
        input_signature=[
                         tf.TensorSpec(shape=None, dtype=tf.float64), 
                         tf.TensorSpec(shape=None, dtype=tf.float64), 
                         tf.TensorSpec(shape=None, dtype=tf.float64), 
                         tf.TensorSpec(shape=(), dtype=tf.int32),
                         ]
                 )
    def predict_tf(self, X, xtr, ytr, k):
        """
        Brute force k nearest neighbors. (calculates distances to all training points)
        """
        # squared distances, between row vectors of a  and row vectors of b, can be calculated with:
        # distances = dot(a, a) - 2 * dot(a, b) + dot(b, b)
        # Brilliant... (a - b)^2 = a^2 - 2ab + b^2
        # Sqrt is NOT needed... brilliant
        # First term broadcast as a column, third term broadcast as a row
        # distances.shape = (xtr.shape[0], X.shape[0])
        # einsum implementatio seems to be faster than above, not by much though.
        # distances = tnp.einsum('ij,ij->i', xtr, xtr)[:, tnp.newaxis] - 2 * tnp.dot(xtr, X.T) + tnp.einsum('ij,ij->i', X, X)[tnp.newaxis, :]
        distances = tnp.sum(xtr * xtr, axis=1)[:, tnp.newaxis] - 2 * tnp.dot(xtr, X.T) + tnp.sum(X * X, axis=1)[tnp.newaxis, :]
        # Using top_k.indices and -distance to emulate numpy argpartition
        nearest_neighbors_index = tf.math.top_k(-distances.T, k=k).indices
        # Distances to k nearest neighbors
        nearest_neighbors_dist = tnp.take_along_axis(distances.T, nearest_neighbors_index, axis=1)
        voting_weights = self.neighbor_weights(nearest_neighbors_dist)
        # K nearest neighbors labels
        labels = tnp.take_along_axis(ytr[tnp.newaxis, :], nearest_neighbors_index, axis=1)
        # finds the label with max weight sum
        predictions = weighted_mode(a=labels, w=voting_weights, axis=1)[0][:, 0]
        return predictions

    def score(self, X, y):
        return self.score_tf(X, y, self.xtr, self.ytr, self.k)

    @tf.function(
        input_signature=[
                         tf.TensorSpec(shape=None, dtype=tf.float64), 
                         tf.TensorSpec(shape=None, dtype=tf.float64), 
                         tf.TensorSpec(shape=None, dtype=tf.float64), 
                         tf.TensorSpec(shape=None, dtype=tf.float64), 
                         tf.TensorSpec(shape=(), dtype=tf.int32),
                         ]
                 )
    def score_tf(self, X, y, xtr, ytr, k):
        predictions = self.predict_tf(X, xtr, ytr, k)
        n = tf.size(y)
        accurate = tnp.sum(predictions == y, dtype=tf.float64)
        accuracy = accurate / n
        return accuracy

    @tf.function(
        input_signature=[
                         tf.TensorSpec(shape=None, dtype=tf.float64), 
                         ]
                 )
    def neighbor_weights(self, dist):
        """
        if user attempts to classify a point that was zero distance from one
        or more training points, those training points are weighted as 1.0
        and the other points as 0.0
        """
        inv_dist = 1. / dist
        inf_mask = tf.math.is_inf(inv_dist)
        inf_row_mask = tf.reduce_any(inf_mask, axis=1)
        indices = tf.where(tf.equal(True, inf_row_mask))
        rows_with_inf = tf.cast(tf.gather_nd(tf.where(inf_mask, 1, 0), indices), tf.float64)
        weights = tf.tensor_scatter_nd_update(inv_dist, indices=indices, updates=rows_with_inf)
        return weights


@tf.function(
    input_signature=[
                    tf.TensorSpec(shape=None, dtype=tf.float64), 
                    tf.TensorSpec(shape=None, dtype=tf.float64), 
                    tf.TensorSpec(shape=(), dtype=tf.int32),
                    ]
            )
def weighted_mode(a, w, axis):
    """
    Based on the beatiful sklearn.utils.extmath.weighted_mode... :)
    """
    assert a.shape == w.shape, 'Array of labels and array of weights must have the same shape.'
    unique_values, _ = tf.unique(tnp.ravel(a))
    
    # This operation reduces the values on the given axis to the value with largest weight sum (thus 1)
    result_shape = (
        tf
        .TensorArray(tf.int32, size=tf.size(a))
        .unstack(tf.shape(a))
        .write(axis, 1)
        .stack()
    )
    # Used to store values with max weight sum
    max_values = tf.zeros(result_shape, dtype=tf.float64)
    # Used to store the max weight sums
    max_w_sum = tf.zeros(result_shape, dtype=tf.float64)

    for value in unique_values:
        # filter only weights where corresponding element in a == value
        w_filtered = tf.where(a == value, w, 0)
        # sums weights (used to determine max)
        w_sum = tnp.expand_dims(tnp.sum(w_filtered, axis), axis)
        # Updates max weight sums and matching values using a mask.
        max_mask = (w_sum > max_w_sum)
        max_values = tf.where(max_mask, value, max_values)
        max_w_sum = tf.where(max_mask, w_sum, max_w_sum)

    return max_values, max_w_sum

# Testing

In [75]:
knn = KNearestNeighborsClassifierTF(k=1)
ks = np.arange(1, 101)

In [76]:
%time knn.fit(xtr, ytr, ks)

CPU times: user 8 s, sys: 385 ms, total: 8.38 s
Wall time: 5.11 s


### Best K after training (from cross validation)






In [59]:
knn.k

28

In [60]:
plot_alpha_scores(ks, scores=knn.k_scores)

In [ ]:
knn.score(xte, yte)

In [ ]:
%timeit knn.predict(xte)

In [ ]:
%timeit knn.score(xte, yte)

## TODO: TF x NP implementations with numerical discrepancies:

Fold | K | TF | NP
-----|---|----|---
1|22|82|81|1
1|28|79|77|2
4|22|87|88|-1
4|28|87|86|1

### Converting `neighbor_weights` from numpy to TF
- numpy mask assigment is not suported in TF.

In [17]:
dist = tf.constant([
    [5., 1., 1., 1.],
    [6., 1., 0, 1.],
    [7., 1., 1., 1.],
    [5.0, 0, 6.8, 0]
], dtype=tf.float64)

In [18]:
x = 1. / dist
x

<tf.Tensor: shape=(4, 4), dtype=float64, numpy=
array([[0.2       , 1.        , 1.        , 1.        ],
       [0.16666667, 1.        ,        inf, 1.        ],
       [0.14285714, 1.        , 1.        , 1.        ],
       [0.2       ,        inf, 0.14705882,        inf]])>

In [19]:
mask = tf.Variable(x, trainable=False, dtype=tf.float64)
mask

<tf.Variable 'Variable:0' shape=(4, 4) dtype=float64, numpy=
array([[0.2       , 1.        , 1.        , 1.        ],
       [0.16666667, 1.        ,        inf, 1.        ],
       [0.14285714, 1.        , 1.        , 1.        ],
       [0.2       ,        inf, 0.14705882,        inf]])>

In [20]:
inf_mask = tf.math.is_inf(x)
inf_row_mask = tf.reduce_any(inf_mask, axis=1)
indices = tf.where(tf.equal(True, inf_row_mask))
indices

<tf.Tensor: shape=(2, 1), dtype=int64, numpy=
array([[1],
       [3]])>

In [21]:
updates = tf.gather_nd(tnp.float64(tf.where(inf_mask, 1, 0)), indices)
updates

<tf.Tensor: shape=(2, 4), dtype=float64, numpy=
array([[0., 0., 1., 0.],
       [0., 1., 0., 1.]])>

In [22]:
new = tf.tensor_scatter_nd_update(mask, indices=indices, updates=updates)
new

<tf.Tensor: shape=(4, 4), dtype=float64, numpy=
array([[0.2       , 1.        , 1.        , 1.        ],
       [0.        , 0.        , 1.        , 0.        ],
       [0.14285714, 1.        , 1.        , 1.        ],
       [0.        , 1.        , 0.        , 1.        ]])>

In [23]:
tf.function
def neighbor_weights(dist):
    inv_dist = 1. / dist
    inf_mask = tf.math.is_inf(inv_dist)
    inf_row_mask = tf.reduce_any(inf_mask, axis=1)
    indices = tf.where(tf.equal(True, inf_row_mask))
    rows_with_inf = tf.gather_nd(tf.where(inf_mask, tnp.float64(1), tnp.float64(0)), indices)
    weights = tf.tensor_scatter_nd_update(inv_dist, indices=indices, updates=rows_with_inf)
    return weights

In [24]:
neighbor_weights(dist)

<tf.Tensor: shape=(4, 4), dtype=float64, numpy=
array([[0.2       , 1.        , 1.        , 1.        ],
       [0.        , 0.        , 1.        , 0.        ],
       [0.14285714, 1.        , 1.        , 1.        ],
       [0.        , 1.        , 0.        , 1.        ]])>

In [25]:
unique, idx = tf.unique(tnp.ravel(yte))
unique

<tf.Tensor: shape=(2,), dtype=float64, numpy=array([6., 5.])>

In [26]:
tf.where(dist > 1, dist, 0)

<tf.Tensor: shape=(4, 4), dtype=float64, numpy=
array([[5. , 0. , 0. , 0. ],
       [6. , 0. , 0. , 0. ],
       [7. , 0. , 0. , 0. ],
       [5. , 0. , 6.8, 0. ]])>

In [26]:
s